In [10]:
using LinearAlgebra, GaussQuadrature, SpecialFunctions

Compute the matrix recurrence coefficients for OPs on the upper branch of the cubic teardrop curve:

$$
y = \frac{1}{2}(1 - x)\sqrt{1 + x}, \qquad x \in [-1, 1].
$$

The inner product is 

$$
\langle f, g \rangle = \int_{-1}^{1} f(x,y)g(x,y) w_{\alpha, \beta} \: \mathrm{d}x, \qquad w_{\alpha, \beta} =  (1-x)^{\alpha}(1 + x)^{\beta}.
$$

Choose $\alpha = \beta = -1/2$. The moments with respect to $w_{-\frac{1}{2},-\frac{1}{2}}$ are known:

$$
\langle 1, x^n \rangle = \frac{(-1)^n + 1}{2} \mathrm{B}(1/2, n/2 + 1/2).
$$

The moments with respect to $yw_{-\frac{1}{2},-\frac{1}{2}}$, i.e., $\langle x^n, y \rangle$ are also required. These are not known explicitly but can be computed with Gauss quadrature.  


In [11]:
d=200
p = Int64(ceil(1 + d*log(10)/log(2)))
setprecision(p)
eps(BigFloat)

6.532100883151301860072294196907045938464429146421064803816689873905821893559499795061788236098601034807838780430910755595368173798221741027664882131395373882593443646144289611768359756745300691779838494e-201

In [12]:
α=BigFloat(-1/2)
β=BigFloat(-1/2)
n = 200
xx2, w2 = jacobi(n, α+BigFloat(1), β+BigFloat(1/2));
w2 = w2/2;

In [13]:
mu2(n) = dot(xx2.^n,w2)
mu(n) = ((-1)^n + 1)/2*beta(BigFloat(1/2),BigFloat(n/2+1/2));

In [14]:
function ipm(a,b)
    
    # Compute the inner product of polynomials on the upper branch of the cubic teardrop using moments
    
    # It is assumed a and b have the same number of coefficients. If not the case, just pad with zeros
    # a contains the contains the ascending order coefficients of the polynomial
    # a0 + a11*x + a12*y + a21x^2 + a22*xy + a31*x^3 + a32*x^2y + ... + an1x^n + an2x^(n-1)y + an+1,1*x^(n+1)
    
    # First compute the coefficients of the product of a and b in the form/convolve the coefficients
    # The convolution of the polynomials in coefficient space takes the form of multiplication by a block Toeplitz matrix
    # c0 + c11x + c12y + c21x^2 + c22xy + c23y^2 + ... + c2n1x^2n + c2n2x^(2n-1)y + c2n3x^(2n-2)y^2 + c2n1,1*x^(2n+1) +
    # c2n1,2*x^(2n)y + c2n2,1*x^(2n+2)
    
    # Then convert the product to the form
    # d0 + d11*x + d12*y + d21x^2 + d22*xy + ... + d2n1,1*x^(2n+1) + d2n1,2*x^(2n)y + d2n2,1*x^(2n+2)
    # using the fact that y^2 = 1/4(1-x)^2(1+x) = 1/4(x^3 - x^2 - x + 1)
    
    # Then use the functions above for the moments to compute the inner product of a and b 
    
    la = length(a)
    lb = length(b)
    if la < lb
        a = vcat(a,zeros(lb-la,1))
    elseif lb < la
        b = vcat(b,zeros(la-lb,1))
    end
    
    n = Int64((length(a)-2)/2)
    c = BigFloat.(zeros(3*(2*n + 1) + 1,1))
    c[1] = a[1]*b[1]
    c[2] = sum(a[2:-1:1].*b[1:2])
    c[3] = sum(a[3:-2:1].*b[1:2:3])
    c[end-3] = sum(a[end:-2:end-2].*b[end-2:2:end])
    c[end-2] = a[end-1]*b[end]
    c[end] = a[end]*b[end]
    for k = 2:n
       c[4+3*(k-2)] = sum(vcat(a[2*k:-2:2],a[1]).*vcat(b[1],b[2:2:2*k])) 
       c[5+3*(k-2)] = sum(vcat(a[2*k+1],a[2*k-1:-1:2],a[1]).*vcat(b[1],b[2:2*k-1],b[2*k+1])) 
       c[6+3*(k-2)] = sum(a[2*k-1:-2:3].*b[3:2:2*k-1])
    end
    k = n+1
    c[4+3*(k-2)] = sum(vcat(a[2*k:-2:2],a[1]).*vcat(b[1],b[2:2:2*k])) 
    c[5+3*(k-2)] = sum(a[2*k-1:-1:2].*b[2:2*k-1]) 
    c[6+3*(k-2)] = sum(a[2*k-1:-2:3].*b[3:2:2*k-1])
    for k = n+2:2*n
        c[4+3*(k-2)] = sum(a[end:-2:2*(k-n-1)].*b[2*(k-n-1):2:end]) 
        c[5+3*(k-2)] = sum(a[end:-1:2*(k-n-1)+1].*b[2*(k-n-1)+1:end]) 
        c[6+3*(k-2)] = sum(a[end-1:-2:2*(k-n-1)+3].*b[2*(k-n-1)+3:2:end-1])
    end
    d = BigFloat.(zeros(2*(2*n+1)+2,1))
    d[1:3]=c[1:3]
    d[end]=c[end]
    d[end-2:end-1]=c[end-3:end-2]
    for k = 2:2*n
       d[2*k:2*k+1] = c[4+3*(k-2):5+3*(k-2)] 
    end
    k = 2
    d[2*k+2] += c[6+3*(k-2)]/4
    d[2*k] -= c[6+3*(k-2)]/4
    d[2*k-2] -= c[6+3*(k-2)]/4
    d[2*k-3] += c[6+3*(k-2)]/4
    for k = 3:2*n
       d[2*k+2] += c[6+3*(k-2)]/4
       d[2*k] -= c[6+3*(k-2)]/4
       d[2*k-2] -= c[6+3*(k-2)]/4
       d[2*k-4] += c[6+3*(k-2)]/4
    end
    ipv = d[1]*mu(0)
    ipv += d[end]*mu(2*n+2)
    for k = 1:2*n+1
        ipv += d[2*k]*mu(k)
        ipv += d[2*k+1]*mu2(k-1)
    end
    ipv
end;

In [15]:
function xmult(p)
   # Give the coefficients of x*p(x,y)
    xp = vcat(0,p[1],0,p[2:end])
end
function ymult(p)
   # Give the coefficients of y*p(x,y)
    n = Int64((length(p)-2)/2)
    yp = BigFloat.(zeros(2*n+2+4))
    yp[3]=p[1]
    yp[5:2:end-1]=p[2:2:end]
    yp[1]+=p[3]/4
    yp[2]-=p[3]/4
    yp[4]-=p[3]/4
    yp[6]+=p[3]/4
    for k = 2:n
        yp[2*k-2]+=p[2*k+1]/4
        yp[2*k]-=p[2*k+1]/4
        yp[2*k+2]-=p[2*k+1]/4
        yp[2*k+4]+=p[2*k+1]/4
    end
    yp
end
nor(p) = sqrt(ipm(p,p));

In [16]:
onesv = BigFloat.([1, 0, 0, 0])
Q0m = 1/nor(onesv)
Q0c = [Q0m,0,0,0]
yQ0c = BigFloat.([0,0,Q0m,0])
A0ym = ipm(yQ0c,Q0c)
u1 = [0, Q0m,0,0]
u2 = [-A0ym*Q0m, 0, Q0m,0]
B0x = [nor(u1) 0]
q11 = u1/B0x[1,1]
B0y = BigFloat.(zeros(1,2))
B0y[1,1] = ipm(u2,q11)
v2 = u2 - B0y[1,1]*q11
B0y[1,2] = nor(v2)
q12 = v2/B0y[1,2]
xq11 = xmult(q11)
xq12 = xmult(q12)
a12 = ipm(xq11,q12)
a22 = ipm(xq12,q12)
A1x = [0 a12;a12 a22]
yq11 = ymult(q11)
yq12 = ymult(q12)
a11 = ipm(yq11,q11)
a12 = ipm(yq11,q12)
a22 = ipm(yq12,q12)
A1y = [a11 a12;a12 a22]
u1 = xq11; u2 = xq12; u3 = yq11; u4 = yq12
u1[1:4] .+= -B0x[1,1]*Q0c                - A1x[1,2]*q12
u2[1:4] .+=               - A1x[2,1]*q11 - A1x[2,2]*q12
u3[1:4] .+= -B0y[1,1]*Q0c - A1y[1,1]*q11 - A1y[1,2]*q12
u4[1:4] .+= -B0y[1,2]*Q0c - A1y[2,1]*q11 - A1y[2,2]*q12
B1x = BigFloat.(zeros(2,3))
B1y = BigFloat.(zeros(2,3))
B1x[2,1] = nor(u2)
q21 = u2/B1x[2,1]
B1y[1,1] = ipm(u3,q21)
v3 = u3;v3[1:6] .+= -B1y[1,1]*q21
B1y[1,2] = nor(v3)
q22 = v3/B1y[1,2]
B1y[2,1] = ipm(u4,q21)
B1y[2,2] = ipm(u4,q22)
v4 = u4- B1y[2,2]*q22; v4[1:6] .+= -B1y[2,1]*q21 
B1y[2,3] = nor(v4)
q23 = v4/B1y[2,3]
B1x[1,1] = ipm(u1,q21)
B1x[1,2] = ipm(u1,q22)
B1x[1,3] = ipm(u1,q23)
xq21 = xmult(q21)
xq22 = xmult(q22)
xq23 = xmult(q23)
a11 = ipm(xq21,q21)
a12 = ipm(xq21,q22)
a13 = ipm(xq21,q23)
a22 = ipm(xq22,q22)
a23 = ipm(xq22,q23)
a33 = ipm(xq23,q23)
A2x = [a11 a12 a13;a12 a22 a23;a13 a23 a33]
yq21 = ymult(q21)
yq22 = ymult(q22)
yq23 = ymult(q23)
a11 = ipm(yq21,q21)
a12 = ipm(yq21,q22)
a13 = ipm(yq21,q23)
a22 = ipm(yq22,q22)
a23 = ipm(yq22,q23)
a33 = ipm(yq23,q23)
A2y = [a11 a12 a13;a12 a22 a23;a13 a23 a33]
q11 = vcat(q11,0,0,0,0)
q12 = vcat(q12,0,0,0,0)
q21 = vcat(q21,0,0)
u1 = xq21; u1[1:8] .+= - B1x[1,1]*q11 - B1x[2,1]*q12 - A2x[1,1]*q21 - A2x[1,2]*q22 - A2x[1,3]*q23
u2 = xq22; u2[1:8] .+= - B1x[1,2]*q11 - B1x[2,2]*q12 - A2x[2,1]*q21 - A2x[2,2]*q22 - A2x[2,3]*q23
u3 = xq23; u3[1:8] .+= - B1x[1,3]*q11 - B1x[2,3]*q12 - A2x[3,1]*q21 - A2x[3,2]*q22 - A2x[3,3]*q23
u4 = yq21; u4[1:8] .+= - B1y[1,1]*q11 - B1y[2,1]*q12 - A2y[1,1]*q21 - A2y[1,2]*q22 - A2y[1,3]*q23
u5 = yq22; u5[1:8] .+= - B1y[1,2]*q11 - B1y[2,2]*q12 - A2y[2,1]*q21 - A2y[2,2]*q22 - A2y[2,3]*q23
u6 = yq23; u6[1:8] .+= - B1y[1,3]*q11 - B1y[2,3]*q12 - A2y[3,1]*q21 - A2y[3,2]*q22 - A2y[3,3]*q23
B2x = BigFloat.(zeros(3,3))
B2y = BigFloat.(zeros(3,3))
B2y[1,1] = nor(u4)
q31 = vcat(u4/B2y[1,1],0,0)
B2y[2,1] = ipm(u5,q31)
v5 = u5 - B2y[2,1]*q31
B2y[2,2] = nor(v5)
q32 = v5/B2y[2,2]
B2y[3,1] = ipm(u6,q31)
B2y[3,2] = ipm(u6,q32)
v6 = u6 - B2y[3,1]*q31 - B2y[3,2]*q32
B2y[3,3] = nor(v6)
q33 = v6/B2y[3,3]
b11 = ipm(u1,q31)
b12 = ipm(u1,q32)
b13 = ipm(u1,q33)
b21 = ipm(u2,q31)
b22 = ipm(u2,q32)
b23 = ipm(u2,q33)
b31 = ipm(u3,q31)
b32 = ipm(u3,q32)
b33 = ipm(u3,q33)
B2x = [b11 b12 b13;b21 b22 b23;b31 b32 b33];

In [17]:
nmax = 40 # lose roughly 5 digits per iteration
Ax = BigFloat.(zeros(3,3,nmax))
Bx = BigFloat.(zeros(3,3,nmax+1))
Ay = BigFloat.(zeros(3,3,nmax))
By = BigFloat.(zeros(3,3,nmax+1))
Bx[:,:,1] = B2x
By[:,:,1] = B2y
qo1 = q21
qo2 = q22
qo3 = q23
qn1 = q31
qn2 = q32
qn3 = q33
qo1 = vcat(qo1,0,0,0,0);qo2 = vcat(qo2,0,0,0,0);qo3 = vcat(qo3,0,0,0,0);
for k = 1:nmax
    xqn1 = xmult(qn1)
    xqn2 = xmult(qn2)
    xqn3 = xmult(qn3)
    yqn1 = ymult(qn1)
    yqn2 = ymult(qn2)
    yqn3 = ymult(qn3)
    a11 = ipm(xqn1,qn1)
    a12 = ipm(xqn1,qn2)
    a13 = ipm(xqn1,qn3)
    a22 = ipm(xqn2,qn2)
    a23 = ipm(xqn2,qn3)
    a33 = ipm(xqn3,qn3)
    Ax[:,:,k] = [a11 a12 a13;a12 a22 a23;a13 a23 a33]
    a11 = ipm(yqn1,qn1)
    a12 = ipm(yqn1,qn2)
    a13 = ipm(yqn1,qn3)
    a22 = ipm(yqn2,qn2)
    a23 = ipm(yqn2,qn3)
    a33 = ipm(yqn3,qn3)
    Ay[:,:,k] = [a11 a12 a13;a12 a22 a23;a13 a23 a33]
    u1 = xqn1;u1[1:end-2].+= - Bx[1,1,k]*qo1 - Bx[2,1,k]*qo2 - Bx[3,1,k]*qo3 - Ax[1,1,k]*qn1 - Ax[1,2,k]*qn2 - Ax[1,3,k]*qn3
    u2 = xqn2;u2[1:end-2].+= - Bx[1,2,k]*qo1 - Bx[2,2,k]*qo2 - Bx[3,2,k]*qo3 - Ax[2,1,k]*qn1 - Ax[2,2,k]*qn2 - Ax[2,3,k]*qn3
    u3 = xqn3;u3[1:end-2].+= - Bx[1,3,k]*qo1 - Bx[2,3,k]*qo2 - Bx[3,3,k]*qo3 - Ax[3,1,k]*qn1 - Ax[3,2,k]*qn2 - Ax[3,3,k]*qn3
    u4 = yqn1;u4[1:end-4].+= - By[1,1,k]*qo1 - By[2,1,k]*qo2 - By[3,1,k]*qo3 - Ay[1,1,k]*qn1 - Ay[1,2,k]*qn2 - Ay[1,3,k]*qn3
    u5 = yqn2;u5[1:end-4].+= - By[1,2,k]*qo1 - By[2,2,k]*qo2 - By[3,2,k]*qo3 - Ay[2,1,k]*qn1 - Ay[2,2,k]*qn2 - Ay[2,3,k]*qn3
    u6 = yqn3;u6[1:end-4].+= - By[1,3,k]*qo1 - By[2,3,k]*qo2 - By[3,3,k]*qo3 - Ay[3,1,k]*qn1 - Ay[3,2,k]*qn2 - Ay[3,3,k]*qn3
    By[1,1,k+1] = nor(u4)
    qm1 = u4/By[1,1,k+1]
    By[2,1,k+1] = ipm(u5,qm1)
    v5 = u5 - By[2,1,k+1]*qm1
    By[2,2,k+1] = nor(v5)
    qm2 = v5/By[2,2,k+1]
    By[3,1,k+1] = ipm(u6,qm1)
    By[3,2,k+1] = ipm(u6,qm2)
    v6 = u6 - By[3,1,k+1]*qm1 - By[3,2,k+1]*qm2
    By[3,3,k+1] = nor(v6)
    qm3 = v6/By[3,3,k+1]
    b11 = ipm(u1,qm1)
    b12 = ipm(u1,qm2)
    b13 = ipm(u1,qm3)
    b21 = ipm(u2,qm1)
    b22 = ipm(u2,qm2)
    b23 = ipm(u2,qm3)
    b31 = ipm(u3,qm1)
    b32 = ipm(u3,qm2)
    b33 = ipm(u3,qm3)
    Bx[:,:,k+1] = [b11 b12 b13;b21 b22 b23;b31 b32 b33]
    qo1.=qn1
    qo2.=qn2
    qo3.=qn3
    qo1 = vcat(qo1,0,0,0,0);qo2 = vcat(qo2,0,0,0,0);qo3 = vcat(qo3,0,0,0,0);
    qn1 = vcat(qn1,0,0,0,0);qn2 = vcat(qn2,0,0,0,0);qn3 = vcat(qn3,0,0,0,0);
    qn1.=qm1
    qn2.=qm2
    qn3.=qm3
end

In [18]:
k = nmax
x(z) = Bx[:,:,k]'/z + Ax[:,:,k-1] + Bx[:,:,k]*z
y(z) = By[:,:,k]'/z + Ay[:,:,k-1] + By[:,:,k]*z
z = exp(im*0)
com = norm(x(z)*y(z) - y(z)*x(z))
alg = norm(y(z)^2 - 1/4*(I - x(z))^2*(I + x(z)))
@show com
@show alg;

com = 3.770031990855851059123311508074010564596718457863197134180521203524430815104555308628317777733490453253131887485519343283296976709096000085746391214166694466338372356773413758350865449651244546880334406e-05
alg = 4.143759111509911313374419041683775652941524734316868543063437335952373130819148536142367372453098354257698688015219648034524876718328250707531899719018544472139824405083651166409773151726066766812415105e-05


The exact large $n$ limits of the matrix recurrence coefficients are:

In [24]:
v = sqrt(2)/64
Bˣ = [0 0 0;1/8 0 0;1/2 -1/8 0]
Bʸ = [v 0 0;-6*v v 0;v 6*v v]
Aˣ = [-1/4 -1/2 -1/8;-1/2 -1/4 1/2;-1/8 1/2 -1/4]
Aʸ = [12*v -v 6*v;-v 12*v v;6*v v 12*v];
x(z) = Bˣ'/z + Aˣ + Bˣ*z
y(z) = Bʸ'/z + Aʸ + Bʸ*z
z = exp(im*0)
com = norm(x(z)*y(z) - y(z)*x(z))
alg = norm(y(z)^2 - 1/4*(I - x(z))^2*(I + x(z)))
@show com
@show alg;

com = 0.0
alg = 9.614813431917819e-17


In [25]:
[norm(Bx[:,:,end] - Bˣ) norm(Ax[:,:,end] - Aˣ) norm(By[:,:,end] - Bʸ) norm(Ay[:,:,end] - Aʸ)]

1×4 Array{BigFloat,2}:
 0.0021807  0.00651893  0.000929169  0.00082194